In [123]:
import numpy as np
import pandas as pd
from datetime import datetime

today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).strftime("%Y%m%d")
today

'20250331'

## Processing raw dataframe

In [162]:
colorder = ['Region/Category', '중분류', '소분류', '사전조건', '테스트 항목', '테스트 스텝', '기대결과']

df = pd.read_csv('data/gateway-validation-tc.csv').drop(columns=["No.", "Purpose"]).fillna(0)
df_billing = df.rename(columns = {'대분류': "중분류", '중분류': "Region/Category"})[colorder]

for index, row in df_billing.iterrows():
    # 대분류 0 -> 중분류
    if row["Region/Category"] == 0:
        replacement = "All Regions"
        df_billing.at[index, "Region/Category"] = replacement
    
    # Language Settings to be under All Regions
    if row["Region/Category"] == "Language Setting":
        df_billing.at[index, "소분류"] = "Language Setting"
        df_billing.at[index, "Region/Category"] = "All Regions"
    
    # if 중분류 == multi profile page, All Regions and move 대분류 -> 소분류
    if row["중분류"] == "Multi Profile Page":
        task = row['Region/Category']
        df_billing.at[index, "소분류"] = task
        df_billing.at[index, 'Region/Category'] = "All Regions"
        
    # if 중분류 == social/sso, All Regions and move 대분류 -> 소분류
    if row["중분류"] == "Social Sign in" or row["중분류"] == "Social Sign up" or row["중분류"] == "SSO":
        platform = row['Region/Category']
        df_billing.at[index, "소분류"] = platform
        df_billing.at[index, 'Region/Category'] = "All Regions"
    
    # Consolidate PLAN crosses, put them under All Regions, PLAN Cross to 중분류
    if row["Region/Category"] == "PLAN Cross":
        start_plan = row.중분류
        end_plan = row.소분류
        df_billing.at[index, "소분류"] = f"{start_plan} > {end_plan}"
        df_billing.at[index, "중분류"] = "PLAN Cross"
        df_billing.at[index, "Region/Category"] = "All Regions"
    
    # Compile all Plan Changes
    if row["중분류"] == "PLAN Downgrade":
        df_billing.at[index, "소분류"] = "Downgrade"
        df_billing.at[index, "중분류"] = "PLAN Change"
    elif row["중분류"] == "PLAN Upgrade":
        df_billing.at[index, "소분류"] = "Upgrade"
        df_billing.at[index, "중분류"] = "PLAN Change"
        
# df_billing[df_billing["Region/Category"] == "All Regions"]
df_regional_consolidation = df_billing[~df_billing["Region/Category"].isin(["US", "Argentina", "Australia", "Spain", "United Kingdom"])]

for index, row in df_regional_consolidation.iterrows():
    if row["소분류"] not in ['Basic Month Single Pass', 'Basic Annual Single Pass', 'Premium Month Single Pass', 'Premium Annual Single Pass']:
        df_regional_consolidation.at[index, "Region/Category"] = "All Regions"
df_regional_consolidation.drop_duplicates()


,Region/Category,중분류,소분류,사전조건,테스트 항목,테스트 스텝,기대결과
8,All Regions,Sign in,0,1. KOCOWA Webpage 진입\n2. 미로그인 상태,Sign in 기능 확인,Sign in 시도,Sign in 진행된다
9,All Regions,Sign Up,0,1. KOCOWA Webpage 진입\n2. 미로그인 상태,Sign up 기능 확인,Sign up 시도,Sign up 진행된다
10,All Regions,Sign Out,0,1. KOCOWA Webpage 진입\n2. 로그인 상태,Sign out 기능 확인,Sign out 시도,Sign out 진행된다
11,All Regions,Sign Out,0,1. KOCOWA App 실행\n2. Sign In\n3. Sign Out,재로그인 가능 확인,해당 계정으로 Sign In,Sign in 진행된다
12,All Regions,Find Password,0,1. KOCOWA Webpage 진입\n2. Sign in page 진입\n3. F...,Find Password 기능 확인,Find Password 시도,Reset Password 이메일 수신된다
...,...,...,...,...,...,...,...
2104,All Regions,Resume PLAN,Premium Monthly,1. PLAN Page 진입\n2. Premium Monthly item 결제\n3...,구독 갱신 일자 노출 확인,구독 갱신 일자 노출 확인,"[MM/DD/YYYY, HH:MM] [AM/PM] [대륙/도시] 노출된다"
2105,All Regions,Resume PLAN,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,Resume Button 선택 시 기능 확인,Resume Button 선택,구독 재개된다
2106,All Regions,Resume PLAN,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,구독 갱신 text 출력 확인,Resume Button 선택,Next Billing Date : text 출력된다
2107,All Regions,Resume PLAN,Premium Annual,1. PLAN Page 진입\n2. Premium Annual item 결제\n3....,구독 갱신 일자 노출 확인,구독 갱신 일자 노출 확인,"[MM/DD/YYYY, HH:MM] [AM/PM] [대륙/도시] 노출된다"


In [165]:
# df_billing[~df_billing["Region/Category"].isin(["US", "Argentina", "Australia", "Spain", "United Kingdom", "Brazil"])].sort_values(by=["Region/Category", "중분류"])
# df_billing[df_billing.중분류 == 0]
# df_billing[df_billing["중분류"] == "PLAN Cross"]
# df_billing[~df_billing["Region/Category"].isin(["US", "Argentina", "Australia", "Spain", "United Kingdom"])]

In [155]:
df_regional = df_billing[df_billing["Region/Category"].isin(["US", "Argentina", "Australia", "Spain", "United Kingdom", "New Zealand", "Brazil"])]
f"Count of test cases by region: US-{df_regional[df_regional["Region/Category"] == "US"]["테스트 항목"].count()}, Argentina-{df_regional[df_regional["Region/Category"] == "Argentina"]["테스트 항목"].count()}, Spain-{df_regional[df_regional["Region/Category"] == "Spain"]["테스트 항목"].count()}, United Kingdom-{df_regional[df_regional["Region/Category"] == "United Kingdom"]["테스트 항목"].count()}, Australia-{df_regional[df_regional["Region/Category"] == "Australia"]["테스트 항목"].count()}, Brazil-{df_regional[df_regional["Region/Category"] == "Brazil"]["테스트 항목"].count()}"

'Count of test cases by region: US-411, Argentina-386, Spain-55, United Kingdom-375, Australia-377, Brazil-493'

In [166]:
df_regional_consolidation["Region/Category"].unique()

array(['All Regions', 'Brazil'], dtype=object)

In [167]:
df_regional_consolidation.중분류.unique()

array(['Sign in', 'Sign Up', 'Sign Out', 'Find Password',
       'Social Sign in', 'Social Sign up', 'Multi Profile Page', 'Home',
       'Content Playble', 'KOAS', 'PLAN Page', 'Checkout Page',
       'Checkout Page\n(Coupon)', 'PLAN Purchasing', 'Cancel PLAN',
       'Renewal Subscription', 'PLAN Change', 'SSO', 'QR Sign in',
       'PLAN Cross', 'Resume PLAN'], dtype=object)

In [168]:
middle_layer_values = df_regional_consolidation.중분류.unique()
for value in middle_layer_values:
    lower_layer_values = df_regional_consolidation[df_regional_consolidation.중분류 == value].소분류.unique()
    print(f"소분류 values for {value}: {lower_layer_values}")
    print()

소분류 values for Sign in: [0]

소분류 values for Sign Up: [0]

소분류 values for Sign Out: [0]

소분류 values for Find Password: [0]

소분류 values for Social Sign in: ['Facebook' 'Google' 'Apple' 'ODK']

소분류 values for Social Sign up: ['Facebook' 'Google' 'Apple' 'ODK']

소분류 values for Multi Profile Page: ['Add Profile' 'Edit Profile' 'Delete Profile' 'Profile Change'
 'PIN Locked']

소분류 values for Home: ['Language Setting']

소분류 values for Content Playble: [0 'KOBACO Policy']

소분류 values for KOAS: [0 'SSO' 'Profile Change' 'KOCOWA ID']

소분류 values for PLAN Page: ['Basic Monthly' 'Basic Annual' 'Premium Monthly' 'Premium Annual'
 'Basic Month Single Pass' 'Basic Annual Single Pass'
 'Premium Month Single Pass' 'Premium Annual Single Pass']

소분류 values for Checkout Page: [0 'Basic Monthly' 'Basic Annual' 'Premium Monthly' 'Premium Annual'
 'Basic Month Single Pass' 'Basic Annual Single Pass'
 'Premium Month Single Pass' 'Premium Annual Single Pass' 'Basic Annuals']

소분류 values for Checkout Page
(Cou

In [169]:
df_regional_consolidation.to_csv(f"data/{today}_processed_billing_test_cases.csv", index=False)